#### Imports

In [ ]:
import os
import re
import json

import numpy as np
import dask.dataframe as dd

import networkx as nx
import plotly.express as px

from itertools import chain
from unidecode import unidecode

## Configurating default behavior

In [ ]:
db_path = '../../data/universidade_federal_do_rio_grande_do_norte_-_ufrn'

In [ ]:
json_path = '../../data/universidade_federal_do_rio_grande_do_norte_-_ufrn.json'

In [ ]:
with open(json_path, 'r') as f:
    json_data = json.load(f)

## Auxiliar functions

In [ ]:
online_resources = list(chain(*[[l['link'] for l in d['recursos'] if 'formato' in l and l['formato'] == 'CSV'] for d in json_data['conjuntoDados']]))
offline_resources = [os.path.join(db_path,p) for p in os.listdir(db_path)]

In [ ]:
db_resources = offline_resources
# db_resources = online_resources

In [ ]:
resources_containing = lambda pattern: [r for r in db_resources if len(re.findall(pattern,r)) and r.endswith('.csv')]
dataframe_containing = lambda pattern='', kwarg={}: dd.read_csv(resources_containing(pattern), sep=';',decimal=',',**kwarg).compute()

## Loading curricular data

In [ ]:
unidades : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing('unidadesacademica'),
    sep=';'
)

In [ ]:
cursos : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing('[^-]cursos-de-grad'),
    sep = ';',
    dtype={
        'convenio_academico': 'object',
        'id_coordenador': 'float64',
    }
)

In [ ]:
estruturas_curriculares : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing('estruturas-curriculares'),
    sep=';',
    dtype={
        'observacao': 'object'
    }
)

In [ ]:
curriculo_componente : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing('curriculo-componente-graduacao'),
    sep=';'
)

In [ ]:
curriculo_componente : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing('componentes-curriculares'),
    sep=';',
    dtype={
        'co_requisito': 'object',
        'equivalencia': 'object',
        'pre_requisito': 'object',
        'procedimentos_avaliacao': 'object',
        'referencias': 'object'
    }
)

In [ ]:
turmas : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing("turmas"),
    sep=';',
    dtype={
        'codigo_turma' : 'object',
        'observacao' : 'object',
        'matricula_docente_externo': 'object',
        'convenio': 'object'
    },
    assume_missing=True
)

In [ ]:
matriculas : dd.DataFrame = dd.read_csv(
    urlpath=resources_containing('matricula'),
    sep=';',
    decimal=',',
    assume_missing=True
)

## Let's play

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input

In [ ]:
estruturas_curriculares['id_curso']

Dask Series Structure:
npartitions=1
    int64
      ...
Name: id_curso, dtype: int64
Dask Name: getitem, 3 graph layers

In [ ]:
app = Dash()
app.layout = html.Div(
    [
        html.H1('Unidade'),
        dcc.Dropdown(unidades['nome'].compute(),value="CAENE",id='dd_unidades'),
        html.H1('Cursos'),
        dcc.Dropdown([],value="",id='dd_cursos'),
        html.H1('Estrutura Curricular'),
        dcc.Dropdown([],value="",id='dd_estrutura'),
    ],
)

@callback(
    Output(component_id='dd_cursos', component_property='options'),
    Input(component_id='dd_unidades', component_property='value')
)
def select_unidade(unidade_name):
    df_unidades_id = unidades[unidades['nome'] == unidade_name]['id_unidade'].values.compute()
    df_cursos   = cursos[cursos['id_unidade_responsavel'].isin(df_unidades_id)].compute()
    
    return df_cursos['nome'].values

@callback(
    Output('dd_estrutura', 'options'),
    Input('dd_cursos','value')
)
def select_curso(curso_nome):
    df_curso_id = cursos[cursos['nome'] == curso_nome]
    return estruturas_curriculares['nome_matriz'].values.compute()

app.run(debug=True)

/home/jose/miniconda3/lib/python3.11/site-packages/dask/dataframe/core.py:3858: UserWarning:

Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.



In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
dir(dash)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [ ]:
dash.__path__

_NamespacePath(['/home/jose/clone/dadosgov/dash'])

In [ ]:
from dash import html

ImportError: cannot import name 'html' from 'dash' (unknown location)

In [ ]:
dash.Dash

AttributeError: module 'dash' has no attribute 'Dash'

In [ ]:
cursos_unidade = cursos[['id_curso','nome','id_unidade_responsavel']].merge(unidades[['id_unidade','nome','sigla']],left_on='id_unidade_responsavel',right_on='id_unidade',suffixes=('_curso','_unidade')).groupby('id_unidade')

True

In [ ]:
px.histogram()

NotImplementedError: DataFrameGroupBy does not allow compute method.Please chain it with an aggregation method (like ``.mean()``) or get a specific group using ``.get_group()`` before calling ``compute()``

In [ ]:
cursos_unidade[['id_curso','nome_unidade']].agg('count').compute()

,id_curso,nome_unidade
id_unidade,,
31011,6,6
443,10,10
351,4,4
445,15,15
442,27,27
30304,1,1
440,6,6
439,18,18
1482,9,9


In [ ]:
cursos

,id_curso,nome,id_coordenador,coordenador,situacao_curso,nivel_ensino,grau_academico,modalidade_educacao,area_conhecimento,tipo_oferta,turno,tipo_ciclo_formacao,municipio,campus,id_unidade_responsavel,unidade_responsavel,website,data_funcionamento,codigo_inep,dou,portaria_reconhecimento,convenio_academico
npartitions=1,,,,,,,,,,,,,,,,,,,,,,
,int64,string,float64,string,string,string,string,string,string,string,string,string,string,string,int64,string,string,string,float64,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
cursos.groupby('id_unidade_responsavel')['id_curso'].count().compute()

id_unidade_responsavel
205       1
284       2
351       4
439      18
440       6
441      12
442      27
443      10
445      15
1482      9
4885      1
4890      4
5965      2
6069      1
7278      1
30304     1
31011     6
Name: id_curso, dtype: int64

In [ ]:
unidades.compute()

,id_unidade,nome,sigla,sigla_academica,id_unidade_responsavel,unidade_responsavel,id_unidade_gestora,unidade_gestora,id_gestora_academica,unidade_gestora_academica,tipo_organizacional,tipo_academica
0,7052,CAENE,CAENE,CAE,248,PRÓ-REITORIA DE ASSUNTOS ESTUDANTIS (PROAE),605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,ASSESSORIA ...,UNIDADE ACADEMICA ESPECIALIZADA
1,440,CENTRO DE BIOCIÊNCIAS,CB,CB,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,440,CENTRO DE BIOCIÊNCIAS,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
2,441,CENTRO DE CIÊNCIAS DA SAÚDE,CCS,CCS,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,441,CENTRO DE CIÊNCIAS DA SAÚDE,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
3,439,CENTRO DE CIÊNCIAS EXATAS E DA TERRA,CCET,CCET,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,439,CENTRO DE CIÊNCIAS EXATAS E DA TERRA,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
4,442,"CENTRO DE CIÊNCIAS HUMANAS, LETRAS E ARTES",CCHLA,CCHLA,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,442,"CENTRO DE CIÊNCIAS HUMANAS, LETRAS E ARTES",605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
5,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS,CCSA,CCSA,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
6,5965,CENTRO DE EDUCAÇÃO,CE,CE,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,5965,CENTRO DE EDUCAÇÃO,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
7,1482,CENTRO DE ENSINO SUPERIOR DO SERIDÓ,CERES,CERES,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,1482,CENTRO DE ENSINO SUPERIOR DO SERIDÓ,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
8,445,CENTRO DE TECNOLOGIA,CT,CT,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,445,CENTRO DE TECNOLOGIA,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,CENTRO ...,CENTRO
9,11,DDP/PROGESP - DIRETORIA DE DESENVOLVIMENTO DE ...,DDP/PROGESP,CCEP,1421,PRÓ-REITORIA DE GESTÃO DE PESSOAS,605,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,11,DDP/PROGESP - DIRETORIA DE DESENVOLVIMENTO DE ...,DIRETORIA ...,UNIDADE ACADEMICA ESPECIALIZADA


In [ ]:
unidades['nome'].compute()

0                                                 CAENE
1                                 CENTRO DE BIOCIÊNCIAS
2                           CENTRO DE CIÊNCIAS DA SAÚDE
3                  CENTRO DE CIÊNCIAS EXATAS E DA TERRA
4            CENTRO DE CIÊNCIAS HUMANAS, LETRAS E ARTES
5                  CENTRO DE CIÊNCIAS SOCIAIS APLICADAS
6                                    CENTRO DE EDUCAÇÃO
7                  CENTRO DE  ENSINO SUPERIOR DO SERIDÓ
8                                  CENTRO DE TECNOLOGIA
9     DDP/PROGESP - DIRETORIA DE DESENVOLVIMENTO DE ...
10                            DIRETORIA DE ENSINO - IMD
11                           ESCOLA AGRÍCOLA DE JUNDIAÍ
12                ESCOLA AGRICOLA DE JUNDIAI - PRONATEC
13                      ESCOLA DE CIÊNCIAS E TECNOLOGIA
14                                     ESCOLA DE MÚSICA
15                                      ESCOLA DE SAÚDE
16    ESCOLA MULTICAMPI DE CIÊNCIAS MÉDICAS DO RIO G...
17    FACULDADE DE CIÊNCIAS DA SAÚDE DO TRAIRI -

In [ ]:
import dash

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
# from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash.dependencies

# code and plot setup
# settings
pd.options.plotting.backend = "plotly"


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       bgcolor='rgba(255, 255, 255, 0.5)', text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

ModuleNotFoundError: No module named 'jupyter_dash'